<a href="https://colab.research.google.com/github/Srinidhi08092006/codesoft/blob/main/arestoverse_large_data_task1_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import files
files.upload()

Saving M05_1000rows_6cols.csv to M05_1000rows_6cols.csv


{'M05_1000rows_6cols.csv': b'CanonicalName,RawValue,Unit,LowerLimit,UpperLimit,SD\nCholesterol,211.76,mg/dL,125.0,200.0,18.75\nGlucose,73.49,mg/dL,70.0,100.0,7.5\nHemoglobin,10.68,g/dL,13.5,17.5,1.0\nVitaminD,34.09,ng/mL,,,\nVitaminD,34.76,ng/mL,,,\nHeartRate,116.48,bpm,60.0,100.0,10.0\nHeartRate,88.16,bpm,60.0,100.0,10.0\nVitaminD,70.86,ng/mL,,,\nHeartRate,122.43,bpm,60.0,100.0,10.0\nGlucose,83.89,mg/dL,70.0,100.0,7.5\nHemoglobin,19.8,g/dL,13.5,17.5,1.0\nGlucose,127.43,mg/dL,70.0,100.0,7.5\nGlucose,119.99,mg/dL,70.0,100.0,7.5\nVitaminD,67.19,ng/mL,,,\nHemoglobin,9.97,g/dL,13.5,17.5,1.0\nHeartRate,119.28,bpm,60.0,100.0,10.0\nCholesterol,203.36,mg/dL,125.0,200.0,18.75\nGlucose,92.19,mg/dL,70.0,100.0,7.5\nVitaminD,26.72,ng/mL,,,\nCholesterol,205.92,mg/dL,125.0,200.0,18.75\nVitaminD,35.94,ng/mL,,,\nCholesterol,216.9,mg/dL,125.0,200.0,18.75\nHemoglobin,19.4,g/dL,13.5,17.5,1.0\nHeartRate,111.46,bpm,60.0,100.0,10.0\nHeartRate,48.46,bpm,60.0,100.0,10.0\nGlucose,56.79,mg/dL,70.0,100.0,7.5\nVit

In [14]:

import numpy as np
import os
import pandas as pd

df = pd.read_csv("M05_1000rows_6cols.csv")  # Use the uploaded file name



# -----------------------------
# Output directory
# -----------------------------
output_dir = "/content/output"
os.makedirs(output_dir, exist_ok=True)

# -----------------------------
# Standardization function
# -----------------------------
def standardize(row):
    raw = row["RawValue"]
    mean, sd = row.get("Mean", np.nan), row.get("SD", np.nan)
    low, high = row.get("RefLow", np.nan), row.get("RefHigh", np.nan)
    flags, z, method = [], np.nan, "UNSTANDARDIZED"

    # Validate reference range
    if pd.isna(low) or pd.isna(high) or high <= low:
        flags.append("MISSING_RANGE")

    # Validate SD
    if not pd.isna(sd) and sd <= 0:
        flags.append("INVALID_SD")

    # Method 1: Z-score using Mean/SD
    if not pd.isna(mean) and not pd.isna(sd) and sd > 0:
        z = (raw - mean) / sd
        method = "Z_SCORE"

    # Method 2: Reference range Z-score
    elif not pd.isna(low) and not pd.isna(high) and high > low:
        mean_rr, sd_rr = (low + high) / 2, (high - low) / 4
        if sd_rr > 0:
            z = (raw - mean_rr) / sd_rr
            method = "REF_RANGE_Z"
        else:
            z = (raw - low) / (high - low)
            method = "MIN_MAX"

    if not pd.isna(z) and abs(z) >= 3:
        flags.append("EXTREME_Z")

    return pd.Series([z, method, ";".join(flags)])

# -----------------------------
# Apply standardization
# -----------------------------
df[["ZScore", "MethodUsed", "Flags"]] = df.apply(standardize, axis=1)

# -----------------------------
# Select output columns
# -----------------------------
final_df = df[["CanonicalName", "RawValue", "Unit", "ZScore", "MethodUsed", "Flags"]]

# -----------------------------
# Save CSV
# -----------------------------
final_df.to_csv(os.path.join(output_dir, "M05_lab_zscores.csv"), index=False)
print("✅ Standardization completed")
print("📁 Files saved in:", output_dir)

✅ Standardization completed
📁 Files saved in: /content/output
